In [0]:
%sql
USE catalog hls_sql_workshop;
CREATE OR REPLACE TABLE hls_sql_workshop.ai.predict_claims_amount_inference_label as (
SELECT
    get_json_object(request, '$.dataframe_records[0].beneficiary_code') AS beneficiary_code,
    CAST(get_json_object(response, '$.predictions[0]') AS DOUBLE) AS prediction,
    b.claim_amount as label,
    request_date,
    databricks_request_id,
    client_request_id,
    request_time,
    status_code,
    sampling_fraction,
    execution_duration_ms,
    request,
    response,
    served_entity_id,
    logging_error_codes,
    requester
FROM
    hls_sql_workshop.ai.predict_claims_amount_payload 
    LEFT JOIN (
SELECT
    a.beneficiary_code as ben_code,
    b.claim_amount
FROM
    cms.gold_dim_beneficiary AS a
INNER JOIN (
    SELECT
        d.beneficiary_code,
        SUM(f.claim_payment_amount) AS claim_amount
    FROM
        cms.gold_fact_patient_claims AS f
    JOIN
        cms.gold_dim_beneficiary AS d ON f.beneficiary_key = d.beneficiary_key
    GROUP BY
        d.beneficiary_code
) AS b ON b.beneficiary_code = a.beneficiary_code
WHERE
    a.__END_AT IS NULL) AS B on b.ben_code =     get_json_object(request, '$.dataframe_records[0].beneficiary_code')
)

In [0]:
%sql
